# Extract MLP's matrix of weights for news dataset

In [1]:
# Download all needed dependencies
# ! pip install git+https://github.com/aimclub/eXplain-NNs &> /dev/null
# ! pip install torchmetrics &> /dev/null
# ! pip install giotto-ph==0.2.2 &> /dev/null
# ! pip install giotto-tda==0.6.0 &> /dev/null
# ! pip install umap-learn==0.5.3 &> /dev/null

In [2]:
# Importing all necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import os
import eXNN.topology
import json

import pandas as pd
pd.set_option('display.max_columns', 70)
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Read preprocessed news dataset
prep_news_df = pd.read_csv('./prep_news_data.csv')
prep_news_df

,rank( timedelta),rank( n_tokens_title),rank( n_tokens_content),rank( n_unique_tokens),rank( n_non_stop_words),rank( n_non_stop_unique_tokens),rank( num_hrefs),rank( num_self_hrefs),num_imgs,num_videos,rank( average_token_length),rank( num_keywords),data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,rank( kw_max_min),rank( kw_avg_min),kw_min_max,kw_max_max,rank( kw_avg_max),kw_min_avg,rank( kw_max_avg),rank( kw_avg_avg),rank( self_reference_min_shares),rank( self_reference_max_shares),rank( self_reference_avg_sharess),weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,rank( LDA_00),rank( LDA_01),rank( LDA_02),rank( LDA_03),rank( LDA_04),rank( global_subjectivity),rank( global_sentiment_polarity),rank( global_rate_positive_words),rank( global_rate_negative_words),rank( rate_positive_words),rank( rate_negative_words),rank( avg_positive_polarity),min_positive_polarity,max_positive_polarity,rank( avg_negative_polarity),rank( min_negative_polarity),max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,3.091313,0.746312,-0.830751,1.171875,-0.859076,1.250329,-0.773700,-0.247319,1.0,0.0,0.060832,-1.077791,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-2.314162,-2.013451,0.0,0.0,-3.091313,0.000000,-3.091313,-3.091313,-0.809729,-0.897243,-0.895353,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.011874,1.107189,0.003762,0.133906,-0.019697,0.834551,-0.299737,0.407371,-0.186943,0.425334,-0.344559,0.257404,0.100000,0.70,-0.840613,-0.309042,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,3.091313,-0.627749,-0.629945,0.636396,-0.682170,1.037852,-1.082209,-0.770293,1.0,0.0,0.874264,-1.654145,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-2.314162,-2.013451,0.0,0.0,-3.091313,0.000000,-3.091313,-3.091313,-1.337803,-1.337726,-1.337803,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.506066,0.481113,0.164789,0.215657,0.071541,-1.206170,0.345029,0.255150,0.038674,0.167738,-0.092096,-0.900984,0.033333,0.70,1.303623,1.355812,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,3.091313,-0.627749,-0.877419,0.346405,-0.977397,-0.284000,-1.082209,-0.770293,1.0,0.0,-0.983430,-0.561454,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-2.314162,-2.013451,0.0,0.0,-3.091313,0.000000,-3.091313,-3.091313,-0.314418,-0.753138,-0.722589,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.628018,-0.102896,-0.194219,-0.319837,1.102533,2.439542,2.017790,1.034661,-0.690612,1.106431,-0.977397,1.564511,0.100000,1.00,-1.637609,-0.868165,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,3.091313,-0.627749,0.309539,-0.344123,0.321434,-0.264207,0.207090,-1.494860,1.0,0.0,-0.947209,-0.086922,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-2.314162,-2.013451,0.0,0.0,-3.091313,0.000000,-3.091313,-3.091313,-1.337803,-1.337726,-1.337803,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.437130,1.172315,0.875191,-0.392816,-0.811837,-0.291810,-0.208737,0.150161,0.574307,-0.284132,0.363172,0.351546,0.136364,0.80,-0.984764,-0.309042,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,3.091313,1.191898,1.253929,-1.200952,1.406604,-1.358432,1.046996,2.268357,20.0,0.0,0.069322,-0.086922,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-2.314162,-2.013451,0.0,0.0,-3.091313,0.000000,-3.091313,-3.091313,-0.768720,1.144183,0.294747,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.314651,-0.220448,-0.531720,-0.626094,1.680026,0.738234,1.686806,1.934743,-0.367261,1.143818,-1.009924,0.671991,0.033333,1.00,0.327633,0.032505,-0.050000,0.454545,0.136364,0.045455,0.136364,505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39639,-3.716757,0.299307,-0.208059,-0.100354,-0.394045,-0.060357,0.207090,1.378593,1.0,1.0,-0.507837,0.343989,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.024409,-0.424676,26900.0,843300.0,0.91118

In [15]:
# Create a Full-Connected Neural Network class
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_dim: int, output_dim: int, dropout: float) -> None:
        super(MultiLayerPerceptron, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(input_dim, 164),
            nn.BatchNorm1d(164),
            nn.LeakyReLU(),
            nn.Dropout(dropout)
        )

        self.layer2 = nn.Sequential(
            nn.Linear(164, 191),
            nn.BatchNorm1d(191),
            nn.LeakyReLU(),
            nn.Dropout(dropout)
        )

        self.layer3 = nn.Sequential(
            nn.Linear(191, 224),
            nn.BatchNorm1d(224),
            nn.LeakyReLU(),
            nn.Dropout(dropout)
        )

        self.layer4 = nn.Sequential(
            nn.Linear(224, 114),
            nn.BatchNorm1d(114),
            nn.LeakyReLU(),   
            nn.Dropout(dropout)
        )

        self.layer5 = nn.Sequential(
            nn.Linear(114, 164),
            nn.BatchNorm1d(164),
            nn.LeakyReLU(),
            nn.Dropout(dropout)
        )

        self.layer6 = nn.Sequential(
            nn.Linear(164, 90),
            nn.BatchNorm1d(90),
            nn.LeakyReLU(),
            nn.Dropout(dropout)
        )

        self.layer7 = nn.Sequential(
            nn.Linear(90, 182),
            nn.BatchNorm1d(182),
            nn.LeakyReLU(),
            nn.Dropout(dropout)
        )

        self.layer8 = nn.Sequential(
            nn.Linear(182, 208),
            nn.BatchNorm1d(208),
            nn.LeakyReLU(),
            nn.Dropout(dropout)
        )

        self.layer9 = nn.Sequential(
            nn.Linear(208, 72),
            nn.BatchNorm1d(72),
            nn.LeakyReLU(),
            nn.Dropout(dropout)
        ) 

        self.layer10 = nn.Sequential(
            nn.Linear(72, 43),
            nn.BatchNorm1d(43),
            nn.LeakyReLU(),
            nn.Dropout(dropout)
        )

        self.layer11 = nn.Sequential(
            nn.Linear(43, 248),
            nn.BatchNorm1d(248),
            nn.LeakyReLU(),
            nn.Dropout(dropout)
        )

        self.layer12 = nn.Sequential(
            nn.Linear(248, 76),
            nn.BatchNorm1d(76),
            nn.LeakyReLU(),
            nn.Dropout(dropout)
        )

        self.layer13 = nn.Sequential(
            nn.Linear(76, output_dim),
            # nn.Sigmoid()
        )

    # Define forward loop for NN
    def forward(self, x):
        x = nn.Flatten()(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        x = self.layer8(x)
        x = self.layer9(x)
        x = self.layer10(x)
        x = self.layer11(x)
        x = self.layer12(x)
        x = self.layer13(x)
        return x

In [16]:
# Define function for calculation RMSE
def root_mean_squared_error(y_true, y_pred):
    mse = torch.mean((y_true - y_pred) ** 2)
    rmse = torch.sqrt(mse)
    return rmse.item()

In [20]:
# Define training and evaluating steps
def train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer, scheduler, epochs, test_data, batch, out_dir):
    home_dir = f'./news_output/{out_dir}/weights_graphs_mlp_DataAmount{batch}/barcodes/'
    os.makedirs(home_dir)
    train_losses = list()
    test_losses = list()
    rmse_scores = list()
    bar_datum = dict()
    bar_evaluation = dict()

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        
        model.eval()
        with torch.no_grad():
            test_loss = 0.0
            all_outputs = list()
            all_targets = list()
            for inputs, targets in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item()
                all_outputs.append(outputs)
                all_targets.append(targets)

            test_loss /= len(test_loader)
            test_losses.append(test_loss)

            all_outputs = torch.cat(all_outputs, dim=0)
            all_targets = torch.cat(all_targets, dim=0)
            rmse_value = root_mean_squared_error(all_targets, all_outputs)
            rmse_scores.append(rmse_value)
        scheduler.step()

        # Applying some TDA methods to study the obtained network
        model = model.to(torch.device('cpu'))
        data = torch.stack([test_data[i][0] for i in range(100)])
        layer_list = ['layer1', 'layer2', 'layer3', 'layer4', 'layer5', 'layer6', 'layer7', 'layer8', 'layer9', 'layer10', 'layer11', 'layer12', 'layer13']
        barcodes = eXNN.topology.get_nn_barcodes(model, data, layers=layer_list, hom_type="standard", coefs_type="2")
        barcodes_data = dict()
        barcodes_eval = dict()
        epoch_dir = home_dir + f'epoch{epoch+1}/'
        os.makedirs(epoch_dir, exist_ok=True)
        for i in range(1, 7):
            barcode = barcodes[f'layer{i}']
            # Save barcodes' info into the JSON
            barcodes_data.setdefault(f'layer{i}', barcode)

            # Obtain the barcode after epoch and saving it
            bplot = eXNN.topology.plot_barcode(barcode)
            save_fig_path = epoch_dir + f'layer{i}_barcode.png'
            bplot.savefig(save_fig_path)

            # Get and save evaluation for barcode into the JSON
            bar_eval = eXNN.topology.evaluate_barcode(barcode)
            barcodes_eval.setdefault(f'layer{i}', bar_eval)

        bar_datum.setdefault(f'epoch{epoch+1}', barcodes_data)
        bar_evaluation.setdefault(f'epoch{epoch+1}', barcodes_eval)

        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, RMSE: {rmse_value:.4f}')

    # Save dictionaries with barcodes' data and evaluation as JSON files
    with open(f'{home_dir}barcode_data.json', 'w', encoding='utf-8') as file:
        json.dump(bar_datum, file, indent=4)
    
    with open(f'{home_dir}barcode_evaluation.json', 'w', encoding='utf-8') as json_eval:
        json.dump(bar_evaluation, json_eval, indent=4)

    # Get model's weights
    model_weights = model.state_dict()

    # Loss and MAPE visualisation
    fig = plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(test_losses, label='Test Loss')
    plt.plot(rmse_scores, label='RMSE')
    plt.title('Loss and RMSE over epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Metric')
    plt.legend()
    plt.grid(True)
    plt.show()

    return fig, model_weights

In [12]:
# Define function for training network on non-full data
def batch_fit(batch_df, epochs: int, batch: int, out_dir: str, dropout: float):
    # Separate data into features and targets
    X = batch_df.iloc[:, :-1].values
    y = batch_df.iloc[:, -1].values
    y = y.reshape(-1, 1)

    # Data normalization
    scaler_X = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)

    scaler_y = MinMaxScaler()
    y_scaled = scaler_y.fit_transform(y)

    # Splitting data into train and test
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.1, random_state=42)

    # Convertation data to PyTorch tensors
    train_features = torch.tensor(X_train, dtype=torch.float32)
    test_features = torch.tensor(X_test, dtype=torch.float32)
    train_targets = torch.tensor(y_train, dtype=torch.float32)
    test_targets = torch.tensor(y_test, dtype=torch.float32)    

    # Creating DataLoader
    train_data = TensorDataset(train_features, train_targets)
    test_data = TensorDataset(test_features, test_targets)

    train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(dataset=test_data, batch_size=32, shuffle=False)

    # Model initialization
    model = MultiLayerPerceptron(59, 1, dropout)

    # Define optimizator and loss function
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.00016799883676835427, weight_decay=0.1)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.96)

    # Perform training
    fig, model_weights = train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer, scheduler, epochs, test_data, batch, out_dir)

    return fig, model_weights

In [21]:
# Create a name for output folder
out_dir = 'exp_0'

# # Create cycle for training network with different amounts of data
# for batch in range(5000, len(prep_news_df), 5000):
#     base_dir = f'./news_output/{out_dir}/weights_graphs_mlp_DataAmount{batch}/'
#     # Getting part of the dataset
#     df_batch = prep_news_df.head(batch)

#     # Training model and getting it's weights
#     fig, model_weights = batch_fit(df_batch, 60, batch, out_dir, 0.25)

#     # Saving weights, loss function and RMSE
#     torch.save(model_weights, f'{base_dir}weights_mlp_DataAmount{batch}.pth')
#     fig.savefig(f'{base_dir}loss_and_rmse_mlp_DataAmount{batch}.png', dpi=300, bbox_inches='tight')
    

# Getting part of the dataset
df_batch = prep_news_df.head(len(prep_news_df))
last_dir = f'./news_output/{out_dir}/weights_graphs_mlp_DataAmount{len(prep_news_df)}/'

# Training model and getting it's weights
fig, model_weights = batch_fit(df_batch, 60, len(prep_news_df), out_dir, 0.25)

# Saving weights and losses
torch.save(model_weights, f'{last_dir}weights_mlp_DataAmount{len(prep_news_df)}.pth')
fig.savefig(f'{last_dir}loss_and_rmse_mlp_DataAmount{len(prep_news_df)}.png', dpi=300, bbox_inches='tight')

/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide



Epoch 1/60, Train Loss: 0.0287, Test Loss: 0.0003, RMSE: 0.0166


/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide



Epoch 2/60, Train Loss: 0.0009, Test Loss: 0.0003, RMSE: 0.0164


/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide



Epoch 3/60, Train Loss: 0.0002, Test Loss: 0.0003, RMSE: 0.0163


/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/pavel/miniconda3/lib/python3.11/site-packages/eXNN/topology/metrics.py:128: RuntimeWarning:

divide by zero encountered in scalar divide



ValueError: zero-size array to reduction operation maximum which has no identity